In [2]:
import pandas as pd
import pyodbc
import pgeocode
from datamart import fix_me

In [3]:
username = 'vigrose'
password_edw = 'Gryffindor~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [20]:
class PPD(pd.DataFrame):

    def __init__(self, filename):
        super().__init__(pd.read_csv(filename))

    def add_ids(self, key_table):
        self = pd.merge(self, key_table, on = 'ME')

    def clean(self):
        self['ME'] = fix_me(ppd.ME)
        self = self.fillna('None')

In [4]:
#Get ppd
ppd = pd.read_csv('../../Data/PPD/ppd_data_20210522.csv')
ppd['ME'] = fix_me(ppd.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,14,16,20,21,42,49,56,57,65,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
x = PPD('../../Data/PPD/ppd_data_20210522.csv')
x.clean()

In [21]:
x.clean()

In [22]:
x

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,00102000013,A,1,2.0,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00102000021,A,1,3.0,MERI K MARTIN MD,MARTIN,MERI,KING,NaN,NaN,...,NC,27617,7408.0,R077,NaN,NaN,NaN,NaN,NaN,NaN
2,00102000030,A,1,1.0,MARK T JACKSON MD,JACKSON,MARK,THOMAS,NaN,NaN,...,NC,28805,1262.0,C082,NaN,NaN,NaN,NaN,NaN,NaN
3,00102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,AZ,85375,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN
4,00102000056,A,1,2.0,RAJIV B GALA MD,GALA,RAJIV,BABULAL,NaN,NaN,...,LA,70115,6949.0,C003,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302611,96501960028,A,1,2.0,ROBERT SAKULANDA MD,SAKULANDA,ROBERT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1302612,96501960036,A,1,2.0,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,NaN,...,OH,45801,4667.0,C005,NaN,NaN,NaN,NaN,NaN,NaN
1302613,96501960044,A,1,2.0,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,NaN,...,VA,24541,2987.0,C002,NaN,NaN,NaN,NaN,NaN,NaN
1302614,96501960052,A,1,2.0,MASIKU A MDALA-GAUSI MD,MDALA-GAUSI,MASIKU,AGATHA,NaN,NaN,...,NC,27157,1.0,C000,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Get party ids
ME_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [6]:
#Get polos
sql_query_polo = \
    '''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    A.CAT_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE,
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS COMM_ID,
    
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL;
    '''
POLOS = pd.read_sql(con=AMAEDW, sql=sql_query_polo)
POLOS.head()

,PARTY_ID,POST_CD_ID,CAT_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,COMM_ID
0,1849882,16269589,NaN,2603 University Blvd,,,Tuscaloosa,AL,35401,2005-12-16 20:20:34.850978,713677
1,1843314,16267642,NaN,633 S Union Ave,,,Ozark,AL,36360,2005-12-16 20:20:34.853611,713743
2,1849885,13850597,NaN,9135 Piscataway Rd Ste 235,,,Clinton,MD,20735,2005-12-16 20:20:34.854390,713765
3,1888588,12193084,NaN,1106 Druid Rd S,,,Clearwater,FL,33756,2005-12-16 20:20:34.858320,713865
4,1798055,16183293,NaN,106 Medical Arts Bldg,,,Anniston,AL,36201,2005-12-16 20:20:34.860548,713910


In [7]:
#Get ppmas
sql_query_ppma = \
    '''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE_CD,
    P.POST_CD AS ZIP,
    A.FROM_DT
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PP'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL
        '''
PPMAS = pd.read_sql(con=AMAEDW, sql=sql_query_ppma)
PPMAS.head()

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT
0,1798316,16267637,590 Davis St E,,,Elba,AL,36323,2005-12-16 20:20:34.847904
1,1903485,15013274,204 W Cherokee Ave,,,Cartersville,GA,30120,2005-12-16 20:20:34.848723
2,1798049,16090212,412 2nd St N,,,Oneonta,AL,35121,2005-12-16 20:20:34.853355
3,1798321,15487429,3800 Reservoir Rd NW,,,Washington,DC,20007,2005-12-16 20:20:34.853865
4,1891617,15012305,6663 Creekview Pl,,,Columbus,GA,31904,2005-12-16 20:20:34.854130


In [16]:
#Merge ppma, polo
addresses = pd.merge(POLOS, PPMAS, on='PARTY_ID', suffixes = ['_POLO','_PPMA'])

In [25]:
#Add party ids and identify missing POLOs
PPD = pd.merge(ppd, party_ids, on = 'ME')
PPD = PPD.fillna('None')
no_polo = PPD[(PPD.POLO_MAILING_LINE_1=='None')&(PPD.POLO_MAILING_LINE_2=='None')]
print(len(no_polo))

331460


In [26]:
#Find filtered
filtered_polos = pd.merge(no_polo, addresses, on='PARTY_ID')
print(len(filtered_polos))

86628


In [27]:
filtered_dpc = filtered_polos[filtered_polos.TOP_CD == 20]

In [39]:
newers = []
for row in filtered_dpc.itertuples():
    newer = False
    if row.FROM_DT_POLO > row.FROM_DT_PPMA:
        newer = True
    newers.append(newer)

In [40]:
filtered_dpc = filtered_dpc.rename(columns ={'STATE_CD':'STATE_PPMA','STATE_x':'STATE','STATE_y':'STATE_POLO'})
filtered_dpc['POLO_NEWER'] = newers
newer = filtered_dpc[filtered_dpc.POLO_NEWER == True]

In [42]:
newer.groupby('ADDRESS_TYPE').count()

,ME,RECORD_ID,UPDATE_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,MAILING_LINE_2,...,COMM_ID,POST_CD_ID_PPMA,ADDR_1_PPMA,ADDR_2_PPMA,ADDR_3_PPMA,CITY_PPMA,STATE_PPMA,ZIP_PPMA,FROM_DT_PPMA,POLO_NEWER
ADDRESS_TYPE,,,,,,,,,,,,,,,,,,,,,
1.0,13307,13307,13307,13307,13307,13307,13307,13307,13307,13307,...,13307,13307,13307,13252,13041,13307,13307,13307,13307,13307
2.0,22200,22200,22200,22200,22200,22200,22200,22200,22200,22200,...,22200,22200,22200,21750,21715,22200,22200,22200,22200,22200
3.0,65,65,65,65,65,65,65,65,65,65,...,65,65,65,65,64,65,65,65,65,65
None,610,610,610,610,610,610,610,610,610,610,...,610,610,610,602,591,610,610,610,610,610


In [53]:
parties = tuple(newer.PARTY_ID)
#Sourcing
sql_query = \
        f'''
        SELECT DISTINCT
        A.PARTY_ID,
        A.POST_CD_ID,
        CC.DESC,
        CC.CATEGORY_CODE
        FROM
        AMAEDW.POST_CD P, AMAEDW.PARTY_ADDR A, AMAEDW.CAT_CD CC, AMAEDW.CONT_PURPOSE_TYPE C
        WHERE
        CC.CAT_CD_ID = A.CAT_CD_ID
        AND
        A.PARTY_ID in {parties}
        AND
        A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
        AND
        C.PURPOSE_USG_CD='PO'
        AND
        C.PURPOSE_CAT_CD='A'
        AND
        A.THRU_DT IS NULL
        '''
sourcing = pd.read_sql(con=AMAEDW, sql=sql_query)

In [69]:
entity_ids = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = '9'
    """
ENTITY_ID = pd.read_sql(con=AMAEDW, sql=entity_ids)

In [74]:
newer = pd.merge(ENTITY_ID, newer, on='PARTY_ID')
entity_list = tuple(newer.ENTITY_ID)

In [76]:
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [78]:
query_3 = \
    f'''
    SELECT DISTINCT
    U.ENTITY_ID,
    U.COMM_ID,
    U.SRC_CAT_CODE
    FROM
    informix.ENTITY_COMM_USG_AT U
    WHERE
    U.COMM_USAGE = 'PO'
    AND
    U.ENTITY_ID in {entity_list}
    AND
    U.END_DT IS NULL
    ;
    '''
aims_check = pd.read_sql(con=informix, sql=query_3)
aims_check.head()

,entity_id,comm_id,src_cat_code
0,1234827,23584424,PHNSURV
1,1234957,23793378,NPI
2,1235529,12142212,DEA
3,1236153,22999259,WEBPHYC
4,1236621,13791447,PHNSURV


In [92]:
newer = pd.merge(aims_check, newer, on='ENTITY_ID')

In [90]:
aims_check['ENTITY_ID'] = aims_check.entity_id.astype(str)

In [107]:
newer = newer.fillna('None')

In [106]:
newer = pd.merge(newer, sourcing, on='PARTY_ID', how='left')

In [109]:
SOURCES  = []
for row in newer.itertuples():
    if row.DESC == 'None':
        source = row.src_cat_code
    else:
        source = row.DESC
    SOURCES.append(source)

In [110]:
newer['SOURCE']=SOURCES

In [149]:
newer.groupby('src_cat_code').count()['entity_id'].to_csv('../../Data/PPD/POLO_Sources.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [43]:
humach = pd.read_csv('../../Data/Humach/humach_updated_since_2018.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
humach['ME']=fix_me(humach.PHYSICIAN_ME_NUMBER)

In [53]:
MERGE = pd.merge(newer, humach, right_on = ['ME','OFFICE_ADDRESS_STATE'], left_on=['ME','STATE_POLO'], suffixes = ['_POLO','_HUMACH'])

In [220]:
MERGE[['ZIP_POLO','OFFICE_ADDRESS_ZIP']]

,ZIP_POLO,OFFICE_ADDRESS_ZIP
0,32610,32610
1,53005,53005
2,45220,45220
3,38801,38801
4,38801,38801
...,...,...
24498,60521,60521
24499,60521,60521
24500,35801,35801
24501,33462,33462


In [46]:
newer['OFFICE_ADDRESS_LINE_2'] = [x.upper().replace('.','') for x in newer.ADDR_1_POLO]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [242]:
true_merge.drop_duplicates('ME').groupby('src_cat_code').count()

,entity_id,comm_id,ENTITY_ID,PARTY_ID,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,...,Source,WSLIVE_SOURCE,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,SOURCE_FILE_DT,VALIDATED
src_cat_code,,,,,,,,,,,,,,,,,,,,,
ADMIT-HOS,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
AFFIL-GRP,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
DEA,150,150,150,150,150,150,150,150,150,150,...,150,150,150,150,150,150,150,150,150,150
E-MAIL,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
GROUP,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
MBSHP-WEB,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
NPI,227,227,227,227,227,227,227,227,227,227,...,227,227,227,227,227,227,227,227,227,227
OLDCC,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
OTHER,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [47]:
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [48]:
def check_match(thing_1, thing_2):
    '''match check'''
    if thing_1 == thing_2:
        return 1
    elif thing_1 in thing_2 or thing_2 in thing_1:
        return 1
    elif fuzz.ratio(thing_1, thing_2) > 60:
        return 1
    else:
        return 0

In [49]:
MERGE = MERGE.fillna('None')

In [52]:
for col in MERGE.columns:
    print(col)

ME
RECORD_ID
UPDATE_TYPE
ADDRESS_TYPE
MAILING_NAME
LAST_NAME
FIRST_NAME
MIDDLE_NAME
SUFFIX_x
MAILING_LINE_1
MAILING_LINE_2
CITY
STATE
ZIP
SECTOR
CARRIER_ROUTE
ADDRESS_UNDELIVERABLE_FLAG
FIPS_COUNTY
FIPS_STATE
PRINTER_CONTROL_CODE
PC_ZIP
PC_SECTOR
DELIVERY_POINT_CODE
CHECK_DIGIT
PRINTER_CONTROL_CODE_2
REGION
DIVISION
GROUP
TRACT
SUFFIX_CENSUS
BLOCK_GROUP
MSA_POPULATION_SIZE
MICRO_METRO_IND
CBSA
CBSA_DIV_IND
MD_DO_CODE
BIRTH_YEAR
BIRTH_CITY
BIRTH_STATE
BIRTH_COUNTRY
GENDER
TELEPHONE_NUMBER
PRESUMED_DEAD_FLAG
FAX_NUMBER
TOP_CD
PE_CD
PRIM_SPEC_CD
SEC_SPEC_CD
MPA_CD
PRA_RECIPIENT
PRA_EXP_DT
GME_CONF_FLG
FROM_DT
TO_DT
YEAR_IN_PROGRAM
POST_GRADUATE_YEAR
GME_SPEC_1
GME_SPEC_2
TRAINING_TYPE
GME_INST_STATE
GME_INST_ID
MEDSCHOOL_STATE
MEDSCHOOL_ID
MEDSCHOOL_GRAD_YEAR
NO_CONTACT_IND
NO_WEB_FLAG
PDRP_FLAG
PDRP_START_DT
POLO_MAILING_LINE_1
POLO_MAILING_LINE_2
POLO_CITY
POLO_STATE
POLO_ZIP
POLO_SECTOR
POLO_CARRIER_ROUTE
MOST_RECENT_FORMER_LAST_NAME
MOST_RECENT_FORMER_MIDDLE_NAME
MOST_RECENT_FORMER_FI

In [56]:
MERGE = MERGE.fillna('None')

In [57]:
corrects =[]
for row in MERGE.itertuples():
    correct = True
    one = row.OFFICE_ADDRESS_LINE_2_POLO
    two = row.OFFICE_ADDRESS_LINE_2_HUMACH
    if check_match(one,two) == 0:
        print(f'{one} is not {two}')
        print(fuzz.ratio(one, two))
        print('')
        correct = False
    corrects.append(correct)

PO BOX 757 is not 216 MARENGO ST
17

PO BOX 757 is not 216 MARANGO ST
17

PO BOX 757 is not 215 MARANGO ST
17

300 E MCBEE AVE is not 67 CREEKSIDE PARK CT
23

17120 2ND AVE S is not 1808 7TH AVE S BDB 652
43

747 52ND ST is not 9300 CAMPUS POINT DR
19

500 S UNIVERSITY AVE is not 6119 MIDTOWN AVE  STE201
27

500 S UNIVERSITY AVE is not 6119 MIDTOWN AVE STE 201
27

1007 GOODYEAR AVE is not 1653 N TEMPLE AVE
41

3316 HIGHWAY 280 is not 321 MEDICAL CENTER DR
32

1613 NW 136TH AVE is not 1300 MICCOSUKEE RD
23

PO BOX 1310 is not 5 MOBILE INFORMEY CIRCLE
23

2915 MISSOURI AVE is not 211 4TH ST
30

1919 STATE ST STE 140 is not 1407 SPRING ST  STE 1
52

2149 W 24TH ST STE B is not 2610 E UNIVERSITY DR
25

13737 NOEL RD STE 1600 is not 504 MEDICAL CENTER BLVD
31

105 N BASCOM AVE STE 201 is not 1000 W CARSON ST
35

350 HAWTHORNE AVE is not 357 34TH ST
43

820 S HIGHWAY 59 is not 820 ROUTE 59
50

2980 SQUALICUM PKWY is not 800 W 5TH AVE
25

1161 21ST AVE S is not 3823 TVC 1301 MEDICAL CENTER DR

3243 GLYNN AVE is not 80 NEWNAN STATION DR STE A
30

3243 GLYNN AVE is not 80 NEWNAN STATION DR STE A
30

100 JACKSON PIKE is not 1515 MAPLE DR
28

2305 GREEN VALLEY RD is not 705 RILEY HOSPITAL DR RM 5867
37

1203 DOGWOOD DR W is not 2250 LEESTOWN RD
24

2700 STANLEY GAULT PKWY STE 129 is not 125 E MASWELL ST
17

2700 STANLEY GAULT PKWY STE 129 is not 125 E MAXWELL ST
17

2700 STANLEY GAULT PKWY STE 129 is not 2601 KENTUCKY AVE
33

1800 MEDICAL CENTER PKWY STE 200 is not SUITE 200
34

2200 CHILDRENS WAY STE 4204 is not CARROLL JUNIOR CHILDRENS HOSPITAL
40

P O BOX 843966 is not 1 HOSPITAL DR
15

555 PIER AVE is not 2333 W MARCH LN  STE A2
23

7777 HENNESSY BLVD is not 3535 BRENTWOOD DR
29

10051 5TH ST N is not 109 TIMBERLACHEN CIR
35

10051 5TH ST N is not 109 TIMBERLACHEN CIR
35

11725 W 112TH ST STE 202 is not 801 N MUR-LEN STE 211
40

2 SAINT MARKS PL is not TWO ST MARKS PLACE STE 110
57

3530 HOUMA BLVD is not 1430 TULANE AVE
47

1305 POST RD STE 310 is not 1305 STE 310 POST RD
6

789 N CLARE AVE is not 1
0

PO BOX 356421 is not 1959 NW PACIFIC ST    BOX 356421
53

7529 STATE RD STE B is not 2123 AUBURN AVE
24

1 HOSPITAL DR DC04600 is not 315 BUSINESS LOOP 70 W
28

240 E 59TH ST 2ND FL is not 2315 BROADWAY
18

240 E 59TH ST 2ND FL is not 2315 BROADWAY
18

50 S STEPHANIE ST STE 101 is not 3475 GS RICHARDS BLVD
30

50 S STEPHANIE ST STE 101 is not PO BOX 663
11

50 S STEPHANIE ST STE 101 is not 3475 GS RICHARDS BLVD
30

1626 N ED CAREY DR is not 6926 WASHITA WAY
41

720 2ND AVE is not 601 S FLOYD ST STE 700
30

2709 LOCHMORE DR is not 4420 LAKE BOONE TRL
40

2709 LOCHMORE DR is not 4420 LAKE BOONE TRL
40

16001 W 9 MILE RD is not 311 MACK AVE
34

5115 BERNARD DR is not 1906 BELLEVIEW AVE SE
33

1050 WELLINGTON AVE is not 2635 N 7TH ST
25

3623 LATROBE DR STE 216 is not 2900 RANDOLPH RD
26

325 DISTEL CIR is not 3883 AIRWAY DR
29

3211 SHANNON RD is not 434 HOSPITAL DR STE 230
21

5505 SIX FORKS RD BLDG 1 is not 1915 MAIL SERVICE CENTER
25

2520 INDEPENDENCE BLVD 

1842 W KINGS HWY is not 1102 W TRENTON RD
42

8845 SIX PINES DR 2ND FL is not 2515 BUSINESS CENTER DR
43

100 HILLCREST MEDICAL BLVD is not 1650 W COLLEGE ST
37

799 E HAMPDEN AVE is not 4545 E 9TH AVE
52

500 W BROADWAY ST is not 24 COACHMAN LANE
30

PO BOX 789 is not 44 ALIANT PKWY
8

30 13TH ST is not PO BOX 1231
29

7101 S PADRE ISLAND DR is not 3240 FORTWORTH ST STE 111
26

2940 E BANNER GATEWAY DR is not 525 CHANDLER VILLAGE DR S
49

5976 W POTTERSTONE CIR is not 3570 W 9000 S
34

1185 W 1650 S is not 5121 COTTONWOOD DR
32

310 COUNTY ROAD 14 is not .
0

1200 N STATE ST is not 2025 MORSE AVE
41

1665 AURORA CT is not 12631 E 17TH AVE
27

77 W UNDERWOOD ST is not 1400 S ORANGE AVE
12

4724 N DAVIS HWY is not 4550 LA JOLLA
28

2362 N OXNARD BLVD is not 4080 LOMA VISTA RD
28

8278 BELLAIRE BLVD STE A is not 3113 ROSS ST
28

200 1ST ST SW is not 855 MANKATO AVE
21

835 S VAN BUREN ST is not 1821 S WEBSTER
44

721 S DOCTORS DR is not 103 OMNI DR
44

900 LANCASTER ST is not 101 BODIN C

4070 E OLYMPIC BLVD is not 742 W HIGHLAND AVE
16

1880 TOWN AND COUNTRY DR STE B101 is not 2909 OREGON CT
30

104 RHODES AVE is not PO BOX 628
25

2555 PONCE DE LEON BLVD FL 4 is not 4200 SUN N LAKE BLVD
50

1040 SIERRA DR is not 7905 CALUMET AVE STE 1
28

16465 SIERRA LAKES PKWY STE 250 is not SWT 250
37

45280 SEELEY DR is not 78120 WILDCAT DR
45

5200 HUMMINGBIRD RD is not 724 S 8TH ST
26

200 W ARBOR DR is not 823 GATEWAY CENTER WAY
22

2390 FARADAY AVE is not 9834 GENESEE AVE
38

160 E MAIN ST is not 1 HOSPITAL DR
23

3580 ATLANTA AVE is not 1500 MOUNT ZION RD
35

3580 ATLANTA AVE is not 1500 MOUNT ZION RD
35

10051 5TH ST N is not 1506 S FRENCH AVE
32

10051 5TH ST N is not 1506 S FRENCH AVE
32

306 STANAFORD RD is not 1256 N EISENHOWER DR
44

455 TOLL GATE RD is not 5750 POST ROAD
47

660 S EUCLID AVE is not 4990 CHILDRENS PLACE
28

660 S EUCLID AVE is not 4523 CLAYTON AVE
44

660 S EUCLID AVE is not 4523 CLAYTON AVE
44

818 RIVERSIDE AVE is not 2620 HORIZON DR SE
23

1400 S COU

In [58]:
MERGE['VALIDATED']=corrects

In [59]:
true_merge = MERGE[MERGE.VALIDATED==True]

In [61]:
true_merge.drop_duplicates('ME')

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX_POLO,MAILING_LINE_1,...,Source,WSLIVE_SOURCE,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,SOURCE_FILE_DT,VALIDATED
0,00102000200,A,1,2,CHARLES R BLAKE MD,BLAKE,CHARLES,RICHARD,None,APT1026,...,Z,UPDATED,06/07/2019,0010200020200032610,102000203522657080,None,OPH,1,None,True
1,00102002351,A,1,1,SUNNY C YUNG MD,YUNG,SUNNY,CHUNG,None,NORTHERN CALIFORNIA HEALTH CARE SYSTEM,...,VT,UPDATED,11/22/2019,00102002351380053005,102002352627544488,None,ID,1,11/01/2019,True
2,00102011678,A,1,2,THOMAS J BENDER MD,BENDER,THOMAS,JOHN,None,None,...,VT,UPDATED,04/02/2021,0010201167334545220,00102011675138724222,None,OM,1,03/02/2021,True
3,00102011848,A,1,1,AMY R DOYLE MD,DOYLE,AMY,ROBINSON,None,None,...,Q,UPDATED,12/25/2020,001020118483038801,00102011846623773000,None,ID,1,12/21/2020,True
5,00102012623,A,1,2,MICHAEL B GUFFEY MD,GUFFEY,MICHAEL,BRADFORD,None,None,...,Z,UPDATED,06/07/2019,0010201262258015010,102012627247736847,None,ID,1,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24269,95701890176,A,1,2,RADE TOMIC MD,TOMIC,RADE,None,None,None,...,Q,UPDATED,03/22/2019,957018901767560611,95701890173129261800,None,PCC,1,None,True
24270,95703910018,A,1,1,NATASA PETRAC MD,PETRAC,NATASA,None,None,BENTON COUNTY HEALTH DEPARTMENT,...,Q,UPDATED,08/23/2019,9570391001401097402,95703910015412428300,None,FM,1,None,True
24271,95704760010,A,1,2,ANETA K DIMOVA MD,DIMOVA,ANETA,KOSTA,None,None,...,VT,UPDATED,11/08/2019,957047600110512401,95704760018453393663,None,CD,1,11/01/2019,True
24273,95709080017,A,1,1,BLERIM ARIFI MD,ARIFI,BLERIM,None,None,JPS HOSPITAL,...,VT,UPDATED,06/19/2020,95709080012406810,95709080012037396959,None,IM,1,06/15/2020,True


In [62]:
true_merge['FILE_DATE'] = pd.to_datetime(true_merge.WSLIVE_FILE_DT)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
goop = true_merge.sort_values('FILE_DATE').drop_duplicates('ME', keep='last')

In [266]:
goop['FILE_YEAR'] = [x.year for x in goop.FILE_DATE]

In [65]:
goop.to_csv('../../Data/POLO_Filter/Filtered_POLOs.csv', index=False)

In [286]:
filtered_dpc[filtered_dpc.POLO_NEWER == False].to_csv('../../Data/POLO_Filter/Filtered_POLOs_Older.csv', index=False)